In [1]:
import requests
import time

url = "http:"

headers = {
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json"
}

payload = {
    "UserIp": "127.0.0.1",
    "Adult": 1,
    "Child": 0,
    "Infant": 0,
    "JourneyType": 1,
    "CabinClass": 1,
    "AirSegments": [
        {
            "Origin": "PNQ",
            "Destination": "VGA",
            "PreferredTime": "2025-08-19T00:00:00"
        }
    ],
    "DirectFlight": True,
    "PreferredCarriers": []
}

num_requests = 5  # Total requests
requests_per_second = 0.1  # 1 request per second

with requests.Session() as session:
    for i in range(num_requests):
        try:
            response = session.post(url, headers=headers, json=payload, timeout=5)
            response.raise_for_status()
            print(f"Request {i+1}: Success - Status {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Request {i+1}: Failed - {e}")
        time.sleep(1 / requests_per_second)


Request 1: Failed - 404 Client Error: Not Found for url: http://13.204.12.26:8000/search
Request 2: Failed - 404 Client Error: Not Found for url: http://13.204.12.26:8000/search
Request 3: Failed - 404 Client Error: Not Found for url: http://13.204.12.26:8000/search


KeyboardInterrupt: 

In [5]:
import requests

url = "http://www.nomadiq.co.in/api/search"

headers = {
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json"
}

# Original Payload
payload = {
    "UserIp": "127.0.0.1",
    "Adult": 1,
    "Child": 0,
    "Infant": 0,
    "JourneyType": 1,
    "CabinClass": 1,
    "AirSegments": [
        {
            "Origin": "IXB",
            "Destination": "VLR",
            "PreferredTime": "2025-07-19T00:00:00"
        }
    ],
    "DirectFlight": True,
    "PreferredCarriers": []
}



try:
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    print("Response 1 JSON:\n",response.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)

Request failed: 500 Server Error: Internal Server Error for url: https://www.nomadiq.co.in/api/search


In [59]:
response_json  = response.json()

In [56]:
import pandas as pd
import json
from datetime import datetime
import numpy as np

def parse_flight_json(json_data):
    """
    Parse flight JSON data and convert to DataFrame for ML model
    """
    flights_data = []
    
    # Extract the results array
    if isinstance(json_data, str):
        data = json.loads(json_data)
    else:
        data = json_data
    
    results = data.get('Result', [[]])[0]  # Get first result array
    
    # Process each flight option
    for flight_idx, flight in enumerate(results):
        segments = flight.get('Segments', [[]])[0]  # Get first segment array
        fare_list = flight.get('FareList', [])
        min_price = flight.get('MinPublishedPrice', 0)
        
        # Process each segment to get flight details
        total_duration = 0
        layover_time = 0
        all_airlines = []
        all_flight_numbers = []
        all_crafts = []
        stops = len(segments) - 1
        
        for seg_idx, segment in enumerate(segments):
            # Extract segment details
            duration = segment.get('Duration', 0)
            total_duration += duration
            layover_time += segment.get('LayoverTime', 0)
            
            # Airline info
            airline_info = segment.get('Airline', {})
            airline_code = airline_info.get('AirlineCode', '')
            airline_name = airline_info.get('AirlineName', '')
            flight_number = airline_info.get('FlightNumber', '')
            
            all_airlines.append(f"{airline_code}-{airline_name}")
            all_flight_numbers.append(f"{airline_code}{flight_number}")
            all_crafts.append(segment.get('Craft', ''))
            
            # Get origin and destination for first and last segments
            if seg_idx == 0:
                origin = segment.get('Origin', {})
                dep_time_str = origin.get('DepartTime', '')
                dep_airport = origin.get('AirportCode', '')
                dep_terminal = origin.get('Terminal', '')
                
            if seg_idx == len(segments) - 1:
                destination = segment.get('Destination', {})
                arr_time_str = destination.get('ArrivalTime', '')
                arr_airport = destination.get('AirportCode', '')
                arr_terminal = destination.get('Terminal', '')
        
        # Parse datetime
        try:
            dep_time = datetime.strptime(dep_time_str, '%Y-%m-%dT%H:%M:%S')
            arr_time = datetime.strptime(arr_time_str, '%Y-%m-%dT%H:%M:%S')
        except:
            continue
            
        # Calculate time features
        days_before_departure = (dep_time.date() - datetime.today().date()).days
        
        # Process each fare option
        for fare_idx, fare in enumerate(fare_list):
            fare_details = fare.get('Fare', {})
            
            # Create a record for each fare option
            record = {
                # Flight identifiers
                # "search_token": data.get('SearchTokenId', ''),
                # "flight_option_id": f"F{flight_idx}",
                # "fare_option_id": f"F{flight_idx}_R{fare_idx}",
                
                # Route information
                "origin": dep_airport,
                "destination": arr_airport,
                "origin_terminal": dep_terminal,
                "destination_terminal": arr_terminal,
                
                # Time features
                "departure_time": dep_time,
                "arrival_time": arr_time,
                "departure_date": dep_time.date(),
                "departure_hour": dep_time.hour,
                "arrival_hour": arr_time.hour,
                # "day_of_week": dep_time.strftime("%A"),
                # "day_of_week_num": dep_time.weekday(),
                # "is_weekend": dep_time.weekday() >= 5,
                "days_before_departure": days_before_departure,
                
                # Flight details
                "airlines": ", ".join(all_airlines),
                "flight_numbers": ", ".join(all_flight_numbers),
                # "aircraft_types": ", ".join(all_crafts),
                # "total_journey_min": int(total_duration),
                # "layover_min": int(layover_time),
                "stops": stops,
                
                # Fare details
                # "fare_type": fare.get('FareType', ''),
                # "is_refundable": fare.get('IsRefundable', False),
                "cabin_class": fare.get('CabinClass', ''),
                
                # Price breakdown
                "base_fare": fare_details.get('BaseFare', 0),
                "tax": fare_details.get('Tax', 0),
                "price_INR": fare_details.get('PublishedPrice', 0),
                "offered_price": fare_details.get('OfferedPrice', 0),
                
                # Baggage info
                "check_in_baggage": extract_baggage_info(fare.get('SeatBaggage', []), 'CheckIn'),
                "cabin_baggage": extract_baggage_info(fare.get('SeatBaggage', []), 'Cabin'),
                "seats_available": extract_seats_available(fare.get('SeatBaggage', [])),
                
                # Additional features
                "airline_remark": fare.get('AirlineRemark', ''),
                "source": fare.get('Source', ''),
                "min_price_for_flight": min_price,
                "price_difference_from_min": fare_details.get('PublishedPrice', 0) - min_price,
                
                # Time-based features
                "is_early_morning": 0 <= dep_time.hour < 6,
                "is_morning": 6 <= dep_time.hour < 12,
                "is_afternoon": 12 <= dep_time.hour < 18,
                "is_evening": 18 <= dep_time.hour < 21,
                "is_night": dep_time.hour >= 21 or dep_time.hour < 6,
                
                # Search metadata
                "search_datetime": datetime.now(),
                "user_ip": data.get('UserIp', '')
            }
            
            flights_data.append(record)
    
    # Convert to DataFrame
    df = pd.DataFrame(flights_data)
    
    # Add derived features
    if not df.empty:
        
        # Advance booking categories
        df['booking_window'] = pd.cut(df['days_before_departure'], 
                                      bins=[0, 7, 14, 30, 60, 365], 
                                      labels=['Last Week', '1-2 Weeks', '2-4 Weeks', '1-2 Months', '2+ Months'])
        
        # Hour categories
        df['departure_time_category'] = pd.cut(df['departure_hour'], 
                                               bins=[0, 6, 12, 18, 24], 
                                               labels=['Night', 'Morning', 'Afternoon', 'Evening'])
    
    return df

def extract_baggage_info(seat_baggage, baggage_type):
    """Extract baggage information from seat baggage array"""
    if seat_baggage and len(seat_baggage) > 0 and len(seat_baggage[0]) > 0:
        return seat_baggage[0][0].get(baggage_type, 'N/A')
    return 'N/A'

def extract_seats_available(seat_baggage):
    """Extract number of seats available"""
    if seat_baggage and len(seat_baggage) > 0 and len(seat_baggage[0]) > 0:
        return seat_baggage[0][0].get('NoOfSeatAvailable', 0)
    return 0

# Example usage
if __name__ == "__main__":
    # Assuming your JSON data is stored in a variable called 'response_json'
    # df = parse_flight_json(response_json)
    
    # For demonstration, create sample data
    
    
    # Parse the data
    df = parse_flight_json(response_json)
    
    # Save to CSV for later use
    # df.to_csv('flight_price_data.csv', index=False)
    
    # Display basic statistics
    # print(f"Total records: {len(df)}")
    # print(f"Unique flights: {df['flight_option_id'].nunique()}")
    # print(f"Price range: ₹{df['price_INR'].min()} - ₹{df['price_INR'].max()}")


In [57]:
df.head()

,origin,destination,origin_terminal,destination_terminal,departure_time,arrival_time,departure_date,departure_hour,arrival_hour,days_before_departure,...,price_difference_from_min,is_early_morning,is_morning,is_afternoon,is_evening,is_night,search_datetime,user_ip,booking_window,departure_time_category
0,HDO,BOM,,2,2025-08-12 11:35:00,2025-08-12 13:45:00,2025-08-12,11,13,28,...,0,False,True,False,False,False,2025-07-15 18:14:24.107663,127.0.0.1,2-4 Weeks,Morning
1,HDO,BOM,,2,2025-08-12 11:35:00,2025-08-12 13:45:00,2025-08-12,11,13,28,...,420,False,True,False,False,False,2025-07-15 18:14:24.107671,127.0.0.1,2-4 Weeks,Morning
2,HDO,BOM,,2,2025-08-12 11:35:00,2025-08-12 13:45:00,2025-08-12,11,13,28,...,3654,False,True,False,False,False,2025-07-15 18:14:24.107676,127.0.0.1,2-4 Weeks,Morning
3,HDO,BOM,,1,2025-08-12 17:35:00,2025-08-12 20:00:00,2025-08-12,17,20,28,...,0,False,False,True,False,False,2025-07-15 18:14:24.107711,127.0.0.1,2-4 Weeks,Afternoon
4,HDO,BOM,,1,2025-08-12 17:35:00,2025-08-12 20:00:00,2025-08-12,17,20,28,...,0,False,False,True,False,False,2025-07-15 18:14:24.107716,127.0.0.1,2-4 Weeks,Afternoon


In [123]:
import pandas as pd
from datetime import datetime
import math

# Conversion rate
EUR_TO_INR = 85.72

# Helper to parse duration like "2h 55m", "1h", or "45m"
def parse_duration(duration_str):
    try:
        h, m = 0, 0
        duration_str = duration_str.strip().lower()
        if 'h' in duration_str:
            h_part = duration_str.split('h')[0].strip()
            h = int(h_part)
            rest = duration_str.split('h')[1]
            if 'm' in rest:
                m = int(rest.replace('m', '').strip())
        elif 'm' in duration_str:
            m = int(duration_str.replace('m', '').strip())
        return h * 60 + m
    except:
        return None

# Extract details
flights_data = []

for f in response_json["FlightOffer"]:
    segments = f["segments"]
    first_segment = segments[0]
    last_segment = segments[-1]

    # Parse start/end timestamps
    dep_time = datetime.strptime(first_segment["departure_time"], "%d.%m.%Y %H:%M")
    arr_time = datetime.strptime(last_segment["arrival_time"], "%d.%m.%Y %H:%M")

    # Parse total journey duration
    duration_min = parse_duration(f["total_duration"])
    stops = f.get("stops", len(segments) - 1)  # fallback if not provided
    cabin_class = f.get("cabin_class", "Economy")

    # Collect segment-wise details
    layover_time_min = 0
    flight_numbers = []
    airlines = []
    segment_durations = []

    for seg in segments:
        flight_numbers.append(seg.get("flight_number"))
        airlines.append(seg.get("airline"))
        segment_durations.append(parse_duration(seg.get("duration", "")))

        # Add layover duration if present
        if seg.get("layover_duration"):
            layover_time_min += parse_duration(seg["layover_duration"])

    flights_data.append({
        "offer_code": f.get("offer_code", ""),
        "departure_time": dep_time,
        "arrival_time": arr_time,
        "airline": airlines,  # List of all airlines
        "flight_number": flight_numbers,  # List of all flight numbers
        "segment_durations": segment_durations,  # Segment durations
        "duration_min": duration_min,
        "duration_hr": round(duration_min / 60, 2),
        "stops": stops,
        "layover_min": layover_time_min,
        "cabin_class": cabin_class,
        "price_EUR": f["total_price"],
        "price_INR": round(f["total_price"] * EUR_TO_INR, 2),
        "day_of_week": dep_time.strftime("%A"),
        "is_weekend": dep_time.weekday() >= 5,
        "departure_hour": dep_time.hour,
        "arrival_hour": arr_time.hour,
        "days_before_departure": (dep_time.date() - datetime.today().date()).days
    })

# Create DataFrame
df = pd.DataFrame(flights_data)

# Optional: Sort by departure time
df.sort_values("departure_time", inplace=True)

# Show sample
print(df.head())


                                           offer_code      departure_time  \
15  H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz... 2025-06-10 02:30:00   
23  H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz... 2025-06-10 02:30:00   
21  H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz... 2025-06-10 04:55:00   
26  H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz... 2025-06-10 06:10:00   
16  H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz... 2025-06-10 07:10:00   

          arrival_time   airline      flight_number segment_durations  \
15 2025-06-10 04:50:00      [AI]          [AI-2577]             [140]   
23 2025-06-10 10:55:00  [AI, AI]  [AI-891, AI-9793]         [170, 90]   
21 2025-06-10 11:05:00  [AI, AI]  [AI-833, AI-9588]        [195, 105]   
26 2025-06-10 08:20:00      [AI]          [AI-2829]             [130]   
16 2025-06-10 09:30:00      [AI]          [AI-2560]             [140]   

    duration_min  duration_hr  stops  layover_min cabin_class  price_EUR  \
15           140      

In [125]:
df.shape

(38, 18)

In [115]:
df['airline']

15    AI
23    AI
21    AI
26    AI
16    AI
29    AI
33    AI
10    AI
36    AI
17    AI
32    AI
13    AI
34    AI
25    AI
27    AI
2     AI
18    AI
0     AI
28    AI
20    AI
3     AI
12    AI
4     AI
9     AI
37    AI
35    AI
1     AI
6     AI
14    AI
5     AI
19    AI
30    AI
31    AI
8     AI
24    AI
22    AI
11    AI
7     AI
Name: airline, dtype: object

In [129]:
df.head(30)

,offer_code,departure_time,arrival_time,airline,flight_number,segment_durations,duration_min,duration_hr,stops,layover_min,cabin_class,price_EUR,price_INR,day_of_week,is_weekend,departure_hour,arrival_hour,days_before_departure
15,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 02:30:00,2025-06-10 04:50:00,[AI],[AI-2577],[140],140,2.33,0,0,Economy,193.10,16552.53,Tuesday,False,2,4,1
23,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 02:30:00,2025-06-10 10:55:00,"[AI, AI]","[AI-891, AI-9793]","[170, 90]",505,8.42,1,245,Economy,219.05,18776.97,Tuesday,False,2,10,1
21,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 04:55:00,2025-06-10 11:05:00,"[AI, AI]","[AI-833, AI-9588]","[195, 105]",370,6.17,1,70,Economy,213.17,18272.93,Tuesday,False,4,11,1
26,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 06:10:00,2025-06-10 08:20:00,[AI],[AI-2829],[130],130,2.17,0,0,Economy,245.11,21010.83,Tuesday,False,6,8,1
16,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 07:10:00,2025-06-10 09:30:00,[AI],[AI-2560],[140],140,2.33,0,0,Economy,193.10,16552.53,Tuesday,False,7,9,1
29,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 08:15:00,2025-06-10 15:35:00,"[AI, AI]","[AI-9697, AI-9703]","[115, 110]",440,7.33,1,215,Economy,302.34,25916.58,Tuesday,False,8,15,1
33,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 09:10:00,2025-06-10 16:40:00,"[AI, AI]","[AI-9517, AI-9591]","[155, 180]",450,7.50,1,115,Economy,348.27,29853.70,Tuesday,False,9,16,1
10,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 09:30:00,2025-06-10 14:15:00,"[AI, AI]","[AI-2571, AI-9897]","[145, 65]",285,4.75,1,75,Economy,179.84,15415.88,Tuesday,False,9,14,1
36,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 09:35:00,2025-06-10 16:35:00,"[AI, AI]","[AI-9652, AI-9661]","[135, 160]",420,7.00,1,125,Economy,394.28,33797.68,Tuesday,False,9,16,1
17,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 09:40:00,2025-06-10 12:00:00,[AI],[AI-2542],[140],140,2.33,0,0,Economy,193.10,16552.53,Tuesday,False,9,12,1


In [124]:
df[df['offer_code'] == 'H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAzMHXUMHZUVlZRhfwdDYysSgBsE1szI2rUlJzanJTUyEKDMEiRtYWhmZ1sC5hgZWpqYgJTUZlSnKampuiTnFqUBzrSzMLYysrAysrEI8rAxNvXWA2AoAFN1QH5AAAAA-b']

,offer_code,departure_time,arrival_time,airline,flight_number,segment_durations,duration_min,duration_hr,stops,layover_min,cabin_class,price_EUR,price_INR,day_of_week,is_weekend,departure_hour,arrival_hour,days_before_departure
0,H4sIAAAAAAAEAHN1MHQwAEMXV586j0iXOiMDI1NdQwNdAz...,2025-06-10 13:40:00,2025-06-11 10:55:00,"[AI, AI]","[AI-2839, AI-9793]","[175, 90]",1275,21.25,1,1010,Economy,110.35,9459.2,Tuesday,False,13,10,1


In [122]:
df[df['flight_number'] == 'AI-9793']

,offer_code,departure_time,arrival_time,airline,flight_number,duration_min,duration_hr,stops,layover_min,cabin_class,price_EUR,price_INR,day_of_week,is_weekend,departure_hour,arrival_hour,days_before_departure


In [126]:
# Remove duplicates based on offer_code
df.drop_duplicates(subset="offer_code", inplace=True)


In [127]:
df.shape

(38, 18)

In [12]:
import pandas as pd

# Filter by airline
flight_df = df[df['airline'] == '6E']

# Sort by 'price_inr' in ascending order (use descending=True for high to low)
flight_df_sorted = flight_df.sort_values(by='price_INR')

# Save to Excel
flight_df_sorted.to_excel("6E_flights_sorted.xlsx", index=False)


In [ ]:
df['stops'].unique()

In [ ]:
import requests
import json

# URL to hit
url = "http://52.66.242.29:8000/api/search"

# Request headers
headers = {
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json"
}

# Updated payload
payload = {
    "adults": 2,
    "children": 1,
    "infants": 1,
    "flight_class": "Econom",
    "stops": None,  # Allow any number of stops
    "airlines": [],  # Include all airlines (if supported)
    "flights": [
        {
            "date": "16.06.2025",
            "iata_from": "BOM",
            "iata_to": "BLR"
        }
    ]
}

# Send the POST request
try:
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad status codes
    response_data = response.json()
    print("Full API Response:\n", json.dumps(response_data, indent=4))
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
